In [7]:
! pip install pandas
! pip install pyspark
! pip install pyarrow
! pip install matplotlib
! pip install seaborn
! pip install scikit-learn

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 60.6/60.6 kB 3.1 MB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.6 MB 17.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.6 MB 29.4 MB/s eta 0:00:01
   ------------------ --------------------- 4.9/10.6 MB 52.5 MB/s eta 0:00:01
   ---------------------- ----------------- 6.0/10.6 MB 35.0 MB/s eta 0:00:01
   ------------------------------------- -- 10.0/10.6 MB 45.8 MB/s eta 0:00:01
   ---------------------------------------- 10.6/10.6 MB 43.5 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   -- ------------------------------------- 3.3/46.2 M

In [151]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [152]:
from pyspark.sql import SparkSession

In [153]:
#Create a Spark session
spark = SparkSession.builder.appName("Tree_Census_NYC").getOrCreate()

In [154]:
#Load the dataset into a Spark DataFrame
df_spark = spark.read.csv("2015_Street_Tree_Census_-_Tree_Data_20240419.csv", header=True, inferSchema=True)

In [155]:
#Convert the Spark DataFrame to a Pandas DataFrame
df = df_spark.toPandas()
df.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,8/27/2015,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,None,None,NoDamage,TreesCount Staff,None,No,No,No,No,No,No,No,No,No,108-005 70 AVENUE,11375,Forest Hills,406,4,Queens,29,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,9/3/2015,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,None,None,Damage,TreesCount Staff,Stones,Yes,No,No,No,No,No,No,No,No,147-074 7 AVENUE,11357,Whitestone,407,4,Queens,19,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,9/5/2015,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,1or2,None,Damage,Volunteer,None,No,No,No,No,No,No,No,No,No,390 MORGAN AVENUE,11211,Brooklyn,301,3,Brooklyn,34,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,9/5/2015,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,1027 GRAND STREET,11211,Brooklyn,301,3,Brooklyn,34,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,8/30/2015,21,0,OnCurb,Alive,Good,Tilia americana,American linden,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,603 6 STREET,11215,Brooklyn,306,3,Brooklyn,39,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


In [156]:
df.describe()

,tree_id,block_id,tree_dbh,stump_diam,postcode,community board,borocode,cncldist,st_assem,st_senate,boro_ct,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
count,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,683788.000000,6.837880e+05,683788.000000,683788.000000,6.837880e+05,683788.000000,677269.000000,677269.000000,6.742290e+05,6.742290e+05
mean,365205.011085,313793.096236,11.279787,0.432463,10916.246044,343.505404,3.358500,29.943181,50.791583,20.615781,3.404914e+06,40.701261,-73.924060,1.005280e+06,194798.424625,30.027330,11957.368422,3.495439e+06,3.413414e+09
std,208122.092902,114839.024312,8.723042,3.290241,651.553364,115.740601,1.166746,14.328531,18.966520,7.390844,1.175863e+06,0.090311,0.123583,3.428505e+04,32902.061114,14.301717,30745.739811,1.193275e+06,1.174892e+09
min,3.000000,100002.000000,0.000000,0.000000,83.000000,101.000000,1.000000,1.000000,23.000000,10.000000,1.000201e+06,40.498466,-74.254965,9.133493e+05,120973.792200,1.000000,1.000000,1.000000e+06,0.000000e+00
25%,186582.750000,221556.000000,4.000000,0.000000,10451.000000,302.000000,3.000000,19.000000,33.000000,14.000000,3.011700e+06,40.631928,-73.980500,9.896578e+05,169515.153700,19.000000,202.000000,3.031991e+06,3.011240e+09
50%,366214.500000,319967.000000,9.000000,0.000000,11214.000000,402.000000,4.000000,30.000000,52.000000,21.000000,4.008100e+06,40.700612,-73.912911,1.008386e+06,194560.252500,30.000000,516.000000,4.020352e+06,4.008560e+09
75%,546170.250000,404624.000000,16.000000,0.000000,11365.000000,412.000000,4.000000,43.000000,64.000000,25.000000,4.103202e+06,40.762228,-73.834910,1.029991e+06,217019.571950,43.000000,1417.000000,4.263123e+06,4.105700e+09
max,722694.000000,999999.000000,450.000000,140.000000,11697.000000,503.000000,5.000000,51.000000,87.000000,36.000000,5.032300e+06,40.912918,-73.700488,1.067248e+06,271894.092100,51.000000,157903.000000,5.515124e+06,5.080500e+09


In [157]:
#Data cleaning
pd.set_option('display.max_columns', None)

In [158]:
#Display the count of 'status' values
df['status'].value_counts()

status
Alive    652173
Stump     17654
Dead      13961
Name: count, dtype: int64

In [159]:
#Display the count of 'health' values
df['health'].groupby(df['status']).value_counts()

status  health
Alive   Good      528850
        Fair       96504
        Poor       26818
Name: count, dtype: int64

In [160]:
#Just keep the 'Alive' trees and remove 'Dead' and 'Stump' trees
df = df[df['status'] == 'Alive'].copy().reset_index(drop=True)

In [161]:
#Map the health statuses to numerical values and create a new column
health_status = {'Good': 1, 'Fair': 2, 'Poor': 3}
df['health_status'] = df['health'].map(health_status)
df.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl,health_status
0,180683,348711,8/27/2015,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,None,None,NoDamage,TreesCount Staff,None,No,No,No,No,No,No,No,No,No,108-005 70 AVENUE,11375,Forest Hills,406,4,Queens,29,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09,2.0
1,200540,315986,9/3/2015,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,None,None,Damage,TreesCount Staff,Stones,Yes,No,No,No,No,No,No,No,No,147-074 7 AVENUE,11357,Whitestone,407,4,Queens,19,27,11,QN49,Whitestone,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09,2.0
2,204026,218365,9/5/2015,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,1or2,None,Damage,Volunteer,None,No,No,No,No,No,No,No,No,No,390 MORGAN AVENUE,11211,Brooklyn,301,3,Brooklyn,34,50,18,BK90,East Williamsburg,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09,1.0
3,204337,217969,9/5/2015,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,1027 GRAND STREET,11211,Brooklyn,301,3,Brooklyn,34,53,18,BK90,East Williamsburg,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09,1.0
4,189565,223043,8/30/2015,21,0,OnCurb,Alive,Good,Tilia americana,American linden,None,None,Damage,Volunteer,Stones,Yes,No,No,No,No,No,No,No,No,603 6 STREET,11215,Brooklyn,306,3,Brooklyn,39,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09,1.0


In [162]:
#Display the count of 'health_status' values
df['health_status'].value_counts()

health_status
1.0    528850
2.0     96504
3.0     26818
Name: count, dtype: int64

In [163]:
#Display the count of 'spc_common' (species) values
df['spc_common'].value_counts()

spc_common
London planetree          87014
honeylocust               64263
Callery pear              58931
pin oak                   53185
Norway maple              34189
littleleaf linden         29742
cherry                    29279
Japanese zelkova          29258
ginkgo                    21024
Sophora                   19338
red maple                 17246
green ash                 16251
American linden           13530
silver maple              12277
sweetgum                  10657
northern red oak           8400
silver linden              7995
American elm               7975
maple                      7080
purple-leaf plum           6879
swamp white oak            6598
crimson king maple         5923
Chinese elm                5345
'Schubert' chokecherry     4888
Japanese tree lilac        4568
eastern redbud             3801
golden raintree            3719
crab apple                 3527
Kentucky coffeetree        3364
willow oak                 3184
dawn redwood               30

In [164]:
#One-hot encoding for categorical features
df = pd.get_dummies(df, columns=['spc_common'], prefix='name', drop_first=True)

In [165]:
#One-hot encoding for categorical features
df['curb_loc'] = df['curb_loc'].map({'OnCurb': 1, 'OffsetFromCurb': 0})
df['steward'] = df['steward'].map({'1or2': 1, '3or4': 2, '4orMore': 3, 'None': 0})
df['guards'] = df['guards'].map({'Harmful': 2, 'Helpful': 3, 'Unsure': 1, 'None': 0})
df['sidewalk'] = df['sidewalk'].map({'Damage': 1, 'NoDamage': 0})

df['root_stone'] = df['root_stone'].map({'Yes': 1, 'No': 0})
df['root_grate'] = df['root_grate'].map({'Yes': 1, 'No': 0})
df['root_other'] = df['root_other'].map({'Yes': 1, 'No': 0})
df['trunk_wire'] = df['trunk_wire'].map({'Yes': 1, 'No': 0})
df['trnk_light'] = df['trnk_light'].map({'Yes': 1, 'No': 0})
df['trnk_other'] = df['trnk_other'].map({'Yes': 1, 'No': 0})
df['brch_light'] = df['brch_light'].map({'Yes': 1, 'No': 0})
df['brch_shoe'] = df['brch_shoe'].map({'Yes': 1, 'No': 0})
df['brch_other'] = df['brch_other'].map({'Yes': 1, 'No': 0})

df.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl,health_status,name_American beech,name_American elm,name_American hophornbeam,name_American hornbeam,name_American larch,name_American linden,name_Amur cork tree,name_Amur maackia,name_Amur maple,name_Atlantic white cedar,name_Atlas cedar,name_Callery pear,name_Chinese chestnut,name_Chinese elm,name_Chinese fringetree,name_Chinese tree lilac,name_Cornelian cherry,name_Douglas-fir,name_English oak,name_European alder,name_European beech,name_European hornbeam,name_Himalayan cedar,name_Japanese hornbeam,name_Japanese maple,name_Japanese snowbell,name_Japanese tree lilac,name_Japanese zelkova,name_Kentucky coffeetree,name_Kentucky yellowwood,name_London planetree,name_Norway maple,name_Norway spruce,name_Ohio buckeye,name_Oklahoma redbud,name_Osage-orange,name_Persian ironwood,name_Schumard's oak,name_Scots pine,name_Shantung maple,name_Siberian elm,name_Sophora,name_Turkish hazelnut,name_Virginia pine,name_arborvitae,name_ash,name_bald cypress,name_bigtooth aspen,name_black cherry,name_black locust,name_black maple,name_black oak,name_black pine,name_black walnut,name_blackgum,name_blue spruce,name_boxelder,name_bur oak,name_catalpa,name_cherry,name_cockspur hawthorn,name_common hackberry,name_crab apple,name_crepe myrtle,name_crimson king maple,name_cucumber magnolia,name_dawn redwood,name_eastern cottonwood,name_eastern hemlock,name_eastern redbud,name_eastern redcedar,name_empress tree,name_false cypress,name_flowering dogwood,name_ginkgo,name_golden raintree,name_green ash,name_hardy rubber tree,name_hawthorn,name_hedge maple,name_holly,name_honeylocust,name_horse chestnut,name_katsura tree,name_kousa dogwood,name_littleleaf linden,name_magnolia,name_maple,name_mimosa,name_mulberry,name_northern red oak,name_pagoda dogwood,name_paper birch,name_paperbark maple,name_pignut hickory,name_pin oak,name_pine,name_pitch pine,name_pond cypress,name_purple-leaf plum,name_quaking aspen,name_red horse chestnut,name_red maple,name_red pine,name_river birch,name_sassafras,name_sawtooth oak,name_scarlet oak,name_serviceberry,name_shingle oak,name_silver birch,name_silver linden,name_silver maple,name_smoketree,name_southern magnolia,name_southern red oak,name_spruce,name_sugar maple,name_swamp white oak,name_sweetgum,name_sycamore maple,name_tartar maple,name_tree of heaven,name_trident maple,name_tulip-poplar,name_two-winged silverbell,name_weeping willow,name_white ash,name_white oak,name_white pine,name_willow oak
0,180683,348711,8/27/2015,3,0,1,Alive,Fair,Acer rubrum,0,0.0,0.0,TreesCount Staff,None,0,0,0,0,0,0,0,0,0,108-005 70 AVENUE,11375,Forest Hills,406,4,Queens,29,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09,2.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,200540,315986,9/3/2015,21,0,1,Alive,Fair,Quercus palustris,0,0.0,1.0,TreesCoun

In [166]:
#Drop rows with null values
pd.set_option('display.max_rows', None)
df.isnull().sum()

tree_id                          0
block_id                         0
created_at                       0
tree_dbh                         0
stump_diam                       0
curb_loc                         0
status                           0
health                           1
spc_latin                        5
steward                          0
guards                           1
sidewalk                         1
user_type                        0
problems                        49
root_stone                       0
root_grate                       0
root_other                       0
trunk_wire                       0
trnk_light                       0
trnk_other                       0
brch_light                       0
brch_shoe                        0
brch_other                       0
address                          0
postcode                         0
zip_city                         0
community board                  0
borocode                         0
borough             

In [167]:
df.drop(df.index[[31282, 407647, 329915]], inplace=True)

In [168]:
df.dropna(inplace=True)

In [169]:
#Checking that all the null rows were removed
df.isnull().sum().sum()

0

In [170]:
print(df.shape)

(642960, 176)


In [171]:
#display the columns
df.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,steward,guards,sidewalk,user_type,problems,root_stone,root_grate,root_other,trunk_wire,trnk_light,trnk_other,brch_light,brch_shoe,brch_other,address,postcode,zip_city,community board,borocode,borough,cncldist,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl,health_status,name_American beech,name_American elm,name_American hophornbeam,name_American hornbeam,name_American larch,name_American linden,name_Amur cork tree,name_Amur maackia,name_Amur maple,name_Atlantic white cedar,name_Atlas cedar,name_Callery pear,name_Chinese chestnut,name_Chinese elm,name_Chinese fringetree,name_Chinese tree lilac,name_Cornelian cherry,name_Douglas-fir,name_English oak,name_European alder,name_European beech,name_European hornbeam,name_Himalayan cedar,name_Japanese hornbeam,name_Japanese maple,name_Japanese snowbell,name_Japanese tree lilac,name_Japanese zelkova,name_Kentucky coffeetree,name_Kentucky yellowwood,name_London planetree,name_Norway maple,name_Norway spruce,name_Ohio buckeye,name_Oklahoma redbud,name_Osage-orange,name_Persian ironwood,name_Schumard's oak,name_Scots pine,name_Shantung maple,name_Siberian elm,name_Sophora,name_Turkish hazelnut,name_Virginia pine,name_arborvitae,name_ash,name_bald cypress,name_bigtooth aspen,name_black cherry,name_black locust,name_black maple,name_black oak,name_black pine,name_black walnut,name_blackgum,name_blue spruce,name_boxelder,name_bur oak,name_catalpa,name_cherry,name_cockspur hawthorn,name_common hackberry,name_crab apple,name_crepe myrtle,name_crimson king maple,name_cucumber magnolia,name_dawn redwood,name_eastern cottonwood,name_eastern hemlock,name_eastern redbud,name_eastern redcedar,name_empress tree,name_false cypress,name_flowering dogwood,name_ginkgo,name_golden raintree,name_green ash,name_hardy rubber tree,name_hawthorn,name_hedge maple,name_holly,name_honeylocust,name_horse chestnut,name_katsura tree,name_kousa dogwood,name_littleleaf linden,name_magnolia,name_maple,name_mimosa,name_mulberry,name_northern red oak,name_pagoda dogwood,name_paper birch,name_paperbark maple,name_pignut hickory,name_pin oak,name_pine,name_pitch pine,name_pond cypress,name_purple-leaf plum,name_quaking aspen,name_red horse chestnut,name_red maple,name_red pine,name_river birch,name_sassafras,name_sawtooth oak,name_scarlet oak,name_serviceberry,name_shingle oak,name_silver birch,name_silver linden,name_silver maple,name_smoketree,name_southern magnolia,name_southern red oak,name_spruce,name_sugar maple,name_swamp white oak,name_sweetgum,name_sycamore maple,name_tartar maple,name_tree of heaven,name_trident maple,name_tulip-poplar,name_two-winged silverbell,name_weeping willow,name_white ash,name_white oak,name_white pine,name_willow oak
0,180683,348711,8/27/2015,3,0,1,Alive,Fair,Acer rubrum,0,0.0,0.0,TreesCount Staff,None,0,0,0,0,0,0,0,0,0,108-005 70 AVENUE,11375,Forest Hills,406,4,Queens,29,28,16,QN17,Forest Hills,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09,2.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,200540,315986,9/3/2015,21,0,1,Alive,Fair,Quercus palustris,0,0.0,1.0,TreesCoun

In [172]:
exclude_columns = ['tree_id', 'block_id', 'created_at', 'stump_diam', 'status', 'health', 'spc_latin', 'user_type', 'problems', 'address', 'zip_city', 'borough', 'nta', 'nta_name', 'state', 'council district', 'census tract', 'bin', 'bbl', 'health_status']
df_excluded_columns = df.drop(columns=exclude_columns)

In [173]:
X = df_excluded_columns
y = df['health_status']

In [174]:
#Test and Train split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [175]:
y_train.value_counts(normalize=True)

health_status
1.0    0.811362
2.0    0.147667
3.0    0.040971
Name: proportion, dtype: float64

In [176]:
#Random Forest
pred_1 = RandomForestClassifier(random_state = 42)
pred_1.fit(X_train, y_train)
accuracy = pred_1.score(X_test, y_test)
print("Accuracy = %g" %accuracy)

Accuracy = 0.826851
